In [9]:
import numpy as np
import pandas as pd

In [1]:
# Very slow for many datapoints.  Fastest for many costs, most readable
def is_pareto_efficient_dumb(costs):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :return: A (n_points, ) boolean array, indicating whether each point is Pareto efficient
    """
    is_efficient = np.ones(costs.shape[0], dtype = bool)
    for i, c in enumerate(costs):
        is_efficient[i] = np.all(np.any(costs[:i]>c, axis=1)) and np.all(np.any(costs[i+1:]>c, axis=1))
    return is_efficient


# Fairly fast for many datapoints, less fast for many costs, somewhat readable
def is_pareto_efficient_simple(costs):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :return: A (n_points, ) boolean array, indicating whether each point is Pareto efficient
    """
    is_efficient = np.ones(costs.shape[0], dtype = bool)
    for i, c in enumerate(costs):
        if is_efficient[i]:
            is_efficient[is_efficient] = np.any(costs[is_efficient]<c, axis=1)  # Keep any point with a lower cost
            is_efficient[i] = True  # And keep self
    return is_efficient


# Faster than is_pareto_efficient_simple, but less readable.
def is_pareto_efficient(costs, return_mask = True):
    """
    Find the pareto-efficient points
    :param costs: An (n_points, n_costs) array
    :param return_mask: True to return a mask
    :return: An array of indices of pareto-efficient points.
        If return_mask is True, this will be an (n_points, ) boolean array
        Otherwise it will be a (n_efficient_points, ) integer array of indices.
    """
    is_efficient = np.arange(costs.shape[0])
    n_points = costs.shape[0]
    next_point_index = 0  # Next index in the is_efficient array to search for
    while next_point_index<len(costs):
        nondominated_point_mask = np.any(costs<costs[next_point_index], axis=1)
        nondominated_point_mask[next_point_index] = True
        is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
        costs = costs[nondominated_point_mask]
        next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
    if return_mask:
        is_efficient_mask = np.zeros(n_points, dtype = bool)
        is_efficient_mask[is_efficient] = True
        return is_efficient_mask
    else:
        return is_efficient

In [6]:
data = pd.read_csv('/home/renansantos/Área de Trabalho/Deb/deb_test/MOEAD_R8/r050n12tw10k4/moead_r8-combined_pareto.csv',
                  header=None)

In [7]:
data

,0,1,2,3,4,5,6,7
0,129928.0,92.0,75.0,4.0,1043.0,149.0,1199.0,0.542987
1,131758.0,110.0,63.0,4.0,1114.0,149.0,1160.0,0.517987
2,159236.0,39.0,129.0,7.0,854.0,137.0,1338.0,0.703416
3,130509.0,81.0,92.0,6.0,1001.0,147.0,1228.0,0.640909
4,129777.0,87.0,110.0,6.0,1015.0,147.0,1219.0,0.606694
...,...,...,...,...,...,...,...,...
4244,156592.0,4.0,126.0,11.0,669.0,128.0,1480.0,0.810929
4245,142005.0,56.0,128.0,8.0,830.0,178.0,1349.0,0.797040
4246,157703.0,9.0,153.0,12.0,742.0,119.0,1411.0,0.849107
4247,157728.0,44.0,138.0,9.0,780.0,181.0,1339.0,0.829204


In [11]:
np.sum(is_pareto_efficient_simple(data.values))

4249